# 层和块

首先，我们回顾一下多层感知机

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[ 0.0074,  0.2389, -0.0171, -0.0982,  0.0142,  0.0451,  0.0927,  0.0901,
          0.1494, -0.0729],
        [-0.0737,  0.2144, -0.0660, -0.0879, -0.0225, -0.0036,  0.1327,  0.0922,
          0.2136, -0.0358]], grad_fn=<AddmmBackward>)

`nn.Sequential`定义了一种特殊的`Module`

自定义块

In [2]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)

    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0717, -0.2565,  0.1002,  0.0634, -0.0921,  0.0888,  0.1963,  0.1157,
         -0.0112,  0.1098],
        [ 0.0184, -0.2341,  0.0798,  0.0780, -0.0420,  0.0676,  0.3624,  0.2192,
         -0.0995,  0.2467]], grad_fn=<AddmmBackward>)

顺序块

In [5]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.0690,  0.0636, -0.0491,  0.2365,  0.0380,  0.1375,  0.2279,  0.1177,
          0.0438,  0.2582],
        [ 0.1523, -0.0009, -0.0119,  0.2484,  0.0624,  0.0513,  0.1844, -0.0362,
          0.1244,  0.1282]], grad_fn=<AddmmBackward>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2308, grad_fn=<SumBackward0>)

混合搭配各种组合块的方法

In [8]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2536, grad_fn=<SumBackward0>)